In [1]:
from craigslist import CraigslistForSale
import json
import time
from datetime import datetime
import os
from tqdm.notebook import tqdm

from lib.listing import Listing

In [2]:
data_dir = 'data'

delay = 3

search_config = {
    "limit": 10_000,
    "category": "sss",
    "search_distance": 30,
    "max_price": 800
}

In [3]:
filters = {
    "has_image": True,
    "search_distance": search_config["search_distance"],
    "max_price": search_config["max_price"],
    "min_price": 1
}

save_dir = os.path.join(data_dir, datetime.now().strftime('%Y%m%d%H%M%S'))
image_dir = os.path.join(save_dir, 'images')
os.makedirs(save_dir)
os.makedirs(image_dir)

# make a file name based on the limit and current time
search = CraigslistForSale(site='westernmass', category=search_config["category"], filters=filters)

print("Approximate number of results:", search.get_results_approx_count())

results = {}
for post in tqdm(search.get_results(sort_by='newest', limit=search_config["limit"]), total=search_config["limit"]):
    start_time = time.time()

    listing = Listing(post)
    listing.fetch_details()
    listing.save_image(image_dir)

    results[listing.id] = listing.to_dict()

    ellapsed_time = time.time() - start_time
    time.sleep(max(0, delay - ellapsed_time))


# save the results to a file
with open(os.path.join(save_dir, "detailed.json"), 'w') as outfile:
    json.dump(results, outfile)

# save the search config to a file
with open(os.path.join(save_dir, "config.json"), 'w') as outfile:
    json.dump(search_config, outfile)

Approximate number of results: 3000


  0%|          | 0/10000 [00:00<?, ?it/s]